# Housing Data

In [222]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

pd.set_option('display.max_columns', 150)

Zipcode boundary datasets
https://github.com/OpenDataDE/State-zip-code-GeoJSON

In [400]:
import_data = False

# Creating Map

In [446]:
cost_cutoff = True
max_price = 1100000
current_data_data_str = '2022-03-31'
target_state = "VA"
target_state_name = "virginia"
state_geojson_path = target_state.lower() + '_' + target_state_name.lower() + '_zip_codes_geo.min.json'

m = folium.Map()

## Walkability
* Rural == 1.2
* Suburban == 8.3
* Historic Main Street = 13.7
* City Center == 17.5

In [447]:
if import_data:
    tract_to_zip = pd.read_excel('DataSets/TRACT_ZIP_122021.xlsx')
    walk_df = pd.read_csv('DataSets/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv')
    walk_df['Tract'] = walk_df.apply(lambda row: create_tract_from_codes(row), axis=1)

In [448]:
def create_tract_from_codes(row):
    state = str(row['STATEFP'])
    county = str(row['COUNTYFP'])
    tract = str(row['TRACTCE'])
    while len(state) < 2:
        state = '0' + state
    while len(county) < 3:
        county = '0' + county
    while len(tract) < 6:
        tract = '0' + tract
    combined = state + county + tract
    return int(combined)

In [449]:
walk_df_with_zip = walk_df.merge(tract_to_zip, how='left', left_on='Tract', right_on='tract')

In [450]:
walk_df_agg_by_zip = pd.DataFrame(walk_df_with_zip.groupby('zip').agg({'NatWalkInd': [np.mean, np.max, np.min, np.std]}))
walk_df_agg_by_zip = walk_df_agg_by_zip['NatWalkInd']
walk_df_agg_by_zip['1stdbelow'] = walk_df_agg_by_zip['mean'] - walk_df_agg_by_zip['std']
walk_df_agg_by_zip['1stdabove'] = walk_df_agg_by_zip['mean'] + walk_df_agg_by_zip['std']
walk_df_agg_by_zip['zip'] = walk_df_agg_by_zip.index.astype(int).astype(str)
walk_df_agg_by_zip = walk_df_agg_by_zip.reset_index(drop=True)

In [456]:
walk_df_agg_by_zip.columns

Index(['mean', 'amax', 'amin', 'std', '1stdbelow', '1stdabove', 'zip'], dtype='object')

In [457]:
folium.Choropleth(geo_data='DataSets/geo_json/' + state_geojson_path,
                  name="Walkability",
                  data=walk_df_agg_by_zip,
                  columns=['zip', 'amax'],
                  key_on='properties.ZCTA5CE10',
                  fill_color='Reds', 
                  fill_opacity=0.7, 
                  line_opacity=0.3,
                  nan_fill_color="Gray",
#                   nan_fill_opacity=1,
                  legend_name='Walkability').add_to(m)

## Zillow House Value Index

In [458]:
if import_data:
    #all sfh by zip
    # zillow_sfh_zip = pd.read_csv('DataSets/Zillow/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv')

    #3br by zip
    zillow_sfh_zip = pd.read_csv('DataSets\Zillow\Zip_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')

    zip_to_lat = pd.read_csv('DataSets/ziptolat.csv')
    zillow_sfh_zip = zillow_sfh_zip.merge(zip_to_lat, how='inner', left_on='RegionName', right_on='ZIP')
    zillow_sfh_zip['ZIP_str'] = zillow_sfh_zip['ZIP'].astype(str)

    zillow_sfh_zip["ZHVI"] = zillow_sfh_zip[current_data_data_str]
    sfh_filtered = zillow_sfh_zip[zillow_sfh_zip["State"] == target_state.upper()][['ZIP', 'ZHVI', 'LAT', 'LNG', 'Metro', 'City', 'ZIP_str']]
    if cost_cutoff:
        sfh_filtered = sfh_filtered[sfh_filtered['ZHVI'] <= max_price]
    sfh_filtered['ZHVI_log'] = np.log(sfh_filtered['ZHVI'])

In [ ]:
folium.Choropleth(geo_data='DataSets/geo_json/' + state_geojson_path,
                  name="ZHVI",
                  data=sfh_filtered,
                  columns=['ZIP_str', 'ZHVI_log'],
                  key_on='properties.ZCTA5CE10',
                  fill_color='Blues', 
                  fill_opacity=0.7, 
                  line_opacity=0.3,
                  nan_fill_color="Gray",
#                   nan_fill_opacity=1,
                  legend_name='ZHVI').add_to(m)

## Adding markers

In [ ]:
create_markers = False

#Maybe use FastMarkerCluster?
if create_markers:
    marker_cluster = MarkerCluster().add_to(m)

    for i in range(sfh_filtered.shape[0]):
        location = [sfh_filtered['LAT'].iloc[i],sfh_filtered['LNG'].iloc[i]]
        tooltip = "Zipcode:{}<br> Location:{}<br> Click for more".format(str(sfh_filtered["ZIP"].iloc[i]), str(sfh_filtered['Metro'].iloc[i]) + ', ' + str(sfh_filtered['City'].iloc[i]))

        folium.Marker(location, # adding more details to the popup screen using HTML
                      popup="""
                      <i>ZHVI: </i> <br> <b>${}</b> <br>
                      """.format(
                        "{:,}".format(sfh_filtered['ZHVI'].iloc[i])
                                ), 
                      tooltip=tooltip).add_to(marker_cluster)

# Display map

In [ ]:
folium.LayerControl().add_to(m)
m